In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/amazon_train.csv')
test = pd.read_csv('/content/drive/MyDrive/amazon_test.csv')

In [ ]:
train.head()

,Unnamed: 0,TITLE,BULLET_POINTS,PRODUCT_LENGTH,x
0,0,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,2125.980000,ArtzFolio Tulip Flowers Blackout Curtain Door ...
1,1,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",393.700000,Marks Spencer Girls Pyjama Sets TCNavy MixYHar...
2,2,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...",748.031495,PRIKNIK Horn Electric Horn Compressor Interior...
3,3,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,787.401574,ALISHAH Womens Cotton Ankle Length Leggings Co...
4,4,The United Empire Loyalists: A Chronicle of th...,@,598.424000,United Empire Loyalists Chronicle Great Migration


In [ ]:
train['x'].fillna(train['TITLE'] , inplace = True)
test['x'].fillna(test['TITLE'] , inplace = True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df = 0.9 , min_df = 2 , stop_words = 'english' , max_features = 5000)
df_train = tfidf_vectorizer.fit_transform(train['x'])
df_test = tfidf_vectorizer.transform(test['x'])

In [ ]:
y_train = train[['PRODUCT_LENGTH']]

In [ ]:
df_train[1:100]

<99x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2082 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Split the data into batches
batch_size = 1000000
num_batches = len(y_train) // batch_size

# Initialize a list to store the decision tree regressors
dtrs = []

for i in range(num_batches):
    # Get the current batch of data
    batch_start = i * batch_size
    batch_end = (i + 1) * batch_size
    X_batch = df_train[batch_start:batch_end]
    y_batch = y_train[batch_start:batch_end]

    # Initialize a new decision tree regressor
    dtr = DecisionTreeRegressor()

    # Train the decision tree regressor on the current batch of data
    dtr.fit(X_batch, y_batch)

    # Add the decision tree regressor to the list
    dtrs.append(dtr)

# Combine the decision tree regressors into a single, final decision tree regressor
final_dtr = DecisionTreeRegressor()
final_dtr.tree_ = np.mean([dtr.tree_ for dtr in dtrs], axis=0).round().astype(int)


In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Set the number of decision trees in the ensemble and the size of the bootstrap samples
batch_size = 1000000
num_batches = len(y_train) // batch_size

# Initialize a list to store the decision tree regressors
dtrs = []

for i in range(num_batches):
    # Get the current batch of data
    batch_start = i * batch_size
    batch_end = (i + 1) * batch_size
    X_batch = df_train[batch_start:batch_end]
    y_batch = y_train[batch_start:batch_end]

    # Initialize a new decision tree regressor
    dtr = DecisionTreeRegressor()

    # Train the decision tree regressor on the current batch of data
    dtr.fit(X_batch, y_batch)

    # Add the decision tree regressor to the list
    dtrs.append(dtr)

# Compute the predictions of the ensemble by averaging the predictions of all decision trees
y_pred = np.mean([dtr.predict(X) for dtr in dtrs], axis=0)


In [ ]:
from gensim.models import Word2Vec

In [ ]:
len(y_train) // 100000

22

In [ ]:
batch_size = 50000
num_batches = len(y_train) // batch_size

# Initialize a list to store the decision tree regressors

dfm = pd.DataFrame()
for i in range(num_batches):
    # Get the current batch of data
    batch_start = i * batch_size
    batch_end = (i + 1) * batch_size
    dff = train[:][batch_start:batch_end]
    dff['newcol'] = dff['x'].apply(lambda x : x.split())
    model = Word2Vec(dff['newcol'], min_count=1 , vector_size = 1000)
    dff['word_embeddings2'] = dff['newcol'].apply(lambda x: np.mean([model.wv[w] for w in x if w in model.wv], axis=0))
    dfm = pd.concat([dfm , dff])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [ ]:

import nltk

class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
xx = train['x'].apply(lambda y : y.split())


In [ ]:
w2v = Word2Vec(train['xx'], min_count=1, size = 1000)

TypeError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df = 0.9 , min_df = 2 , stop_words = 'english' , max_features = 10000)
df_train = tfidf_vectorizer.fit_transform(train['x'])
df_test = tfidf_vectorizer.transform(test['x'])

KeyboardInterrupt: ignored

In [ ]:
y_train = train[['PRODUCT_LENGTH']]

In [ ]:
import gensim.downloader as api
# Load the pre-trained GloVe model with 100 dimensions
glove_model = api.load('glove-wiki-gigaword-100')
def transform_text_to_vectors(text):
    words = text.split()
    vector = np.zeros((100,))
    count = 0
    for word in words:
        if word in glove_model:
            vector += glove_model[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

In [ ]:
X_train = train['x'].apply(transform_text_to_vectors)
X_test = test['x'].apply(transform_text_to_vectors)

In [ ]:
X_train = X_train.tolist()
X_test = X_test.tolist()

In [ ]:
from sklearn.svm import LinearSVR
svr = LinearSVR(max_iter = 50000 , tol = 0.00001 , loss = 'squared_epsilon_insensitive')

In [ ]:
svr.fit(X_train , y_train)
pred = svr.predict(X_test)
predicted = svr.predict(X_train)
pred = pd.DataFrame(pred)

NameError: ignored

In [ ]:
from keras.models import Sequential
import keras
from keras.layers import Dense , Dropout , LSTM , Bidirectional , Embedding

model = keras.Sequential()
model.add(keras.layers.Input(shape=(None, 10000)))
model.add(keras.layers.LSTM(256, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(256, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(128, activation='relu', return_sequences=False))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(1, activation='relu'))
model.compile(loss='rmse', optimizer='adam', metrics=['mae'])

In [ ]:
model.fit(df_train , y_train , epochs = 10)

ValueError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df = 0.9 , min_df = 2 , stop_words = 'english' , max_features = 1000)
df_train = tfidf_vectorizer.fit_transform(train['x'])
df_test = tfidf_vectorizer.fit_transform(test['x'])

In [ ]:
y_train = train[['PRODUCT_LENGTH']]

In [ ]:
from sklearn.svm import SVR
svr = SVR()

In [ ]:
svr.fit(df_train , y_train)
pred = svr.predict(df_test)
# predicted = dtr.predict(df_train)
pred = pd.DataFrame(pred)

In [ ]:
from sklearn import metrics
(1-metrics.mean_absolute_percentage_error(y_train,predicted))

In [ ]:
pred.to_csv(r'submit.csv')